In [1]:
import pandas as pd
import os 
from tqdm import tqdm
import numpy as np

In [2]:
def subs(x):
    if x<0:
        return -x
    elif x>0:
        return 0
    else:
        return x

In [3]:
wages=pd.read_excel('0919 shoes data and codebook.xlsx',sheet_name='wages')
prod=pd.read_excel('0919 shoes data and codebook.xlsx',sheet_name='prod')
wages.head(3)

,conveyor,month,all_salary,workers,avg_pay,attrition,cus1,cus1ps,cus2,cus2ps,cus3,cus3ps,cus4,cus4ps,cus5,cus5ps,allps
0,A-1,1,1119218.3,126,8882.684921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A-1,2,1059743.7,114,9295.997368,9.52381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A-1,3,1042766.2,117,8912.531624,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
allConveyors=wages.conveyor.unique()

## `allcus`, `cus*`, `cus*ps`, shares, avgPrices

In [5]:
prod['totalSale']=prod.priceUSD * prod.pairs
wages.attrition=wages.groupby('conveyor').workers.pct_change().apply(subs)
for i in range(0,3):
        wages[f'cus{i+1}price']=pd.NA

for i,r in wages.iterrows():
    conveyor=r.conveyor
    month=r.month
    prod_filtered=prod[(prod.conveyor==conveyor)&(prod.extenddlymonth==month)]
    info=prod_filtered.groupby('buyername')[['totalSale','pairs']].agg('sum').sort_values('pairs',ascending=0).to_dict()
    totalSale=info['totalSale']
    totalPairs=info['pairs']
    avgPrices=dict(zip(totalSale.keys(), [totalSale[x]/totalPairs[x] for x in totalSale.keys()]))
    try:
        totalAvgPrice=sum(totalSale.values())/sum(totalPairs.values())
    except ZeroDivisionError:
        totalAvgPrice=pd.NA

    try:
            minAvgPrice=min(avgPrices.values())
            maxAvgPrice=max(avgPrices.values())
    except ValueError:
            minAvgPrice=pd.NA
            maxAvgPrice=pd.NA  
            
    wages.loc[i,'allps']=sum(totalPairs.values())
    for rank in range(0,6):
        try:
            wages.loc[i,f'cus{rank+1}']=list(totalPairs.keys())[rank]
            wages.loc[i,f'cus{rank+1}ps']=totalPairs[list(totalPairs.keys())[rank]]
        except:
            continue
    
    # avgPrices
    for ii in range(0,3):
        try:
            wages.loc[i, f'cus{ii+1}price']=list(avgPrices.values())[ii]
        except:
            continue

    wages.loc[i, "avgprice"]=totalAvgPrice
    wages.loc[i, "minprice"]=minAvgPrice
    wages.loc[i, "maxprice"]=maxAvgPrice

    # numbuyers,numstyles,numartno, numbatches
    wages.loc[i, "numbuyers"]=prod_filtered.buyername.nunique()
    wages.loc[i, "numstyles"]=prod_filtered.stylecode.nunique()
    wages.loc[i, "numartno"]=prod_filtered.artno.nunique()
    wages.loc[i, "numbatches"]=prod_filtered.batch.nunique()

# share
wages['cus1share']=(wages.cus1ps/wages.allps)*100
wages

C:\Users\JselZ\AppData\Local\Temp\ipykernel_11596\985906724.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Curfu' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  wages.loc[i,f'cus{rank+1}']=list(totalPairs.keys())[rank]
C:\Users\JselZ\AppData\Local\Temp\ipykernel_11596\985906724.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Bull' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  wages.loc[i,f'cus{rank+1}']=list(totalPairs.keys())[rank]
C:\Users\JselZ\AppData\Local\Temp\ipykernel_11596\985906724.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Rhino' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  wag

,conveyor,month,all_salary,workers,avg_pay,attrition,cus1,cus1ps,cus2,cus2ps,...,avgprice,minprice,maxprice,numbuyers,numstyles,numartno,numbatches,cus6,cus6ps,cus1share
0,A-1,1,1119218.3,126,8882.684921,NaN,Curfu,14562.0,Bull,5832.0,...,16.851480,15.208230,19.629999,4.0,13.0,9.0,20.0,NaN,NaN,58.665700
1,A-1,2,1059743.7,114,9295.997368,0.095238,Rhino,15436.0,Curfu,4114.0,...,17.616919,13.684322,18.878842,3.0,16.0,9.0,33.0,NaN,NaN,73.792906
2,A-1,3,1042766.2,117,8912.531624,0.000000,Curfu,18015.0,Rhino,12421.0,...,17.449033,16.420757,19.023271,3.0,14.0,8.0,29.0,NaN,NaN,57.281399
3,A-1,4,1003444.4,116,8650.382759,0.008547,Rhino,29389.0,Curfu,2766.0,...,20.026005,17.572722,21.450001,3.0,17.0,9.0,64.0,NaN,NaN,91.091963
4,A-1,5,1042253.3,117,8908.147863,0.000000,Rhino,30698.0,NaN,NaN,...,21.460913,21.460913,21.460913,1.0,10.0,5.0,28.0,NaN,NaN,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,C-6,9,298253.1,38,7848.765789,0.173913,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
126,C-7,6,358132.7,48,7461.097917,NaN,Crown,6681.0,Rhino,144.0,...,16.006550,15.900431,20.930000,2.0,5.0,5.0,5.0,NaN,NaN,97.890110
127,C-7,7,384174.2,49,7840.289796,0.000000,Crown,8390.0,Rhino,906.0,...,18.477852,17.407154,21.442583,3.0,15.0,9.0,19.0,NaN,NaN,90.137516
128,C-7,8,400679.3,52,7705.371154,0.000000,Crown,8760.0,Sparky,1500.0,...,18.735139,17.407154,18.962534,2.0,12.0,6.0,15.0,NaN,NaN,85.380117


In [6]:
wages[wages.cus1ps==30698]

,conveyor,month,all_salary,workers,avg_pay,attrition,cus1,cus1ps,cus2,cus2ps,...,avgprice,minprice,maxprice,numbuyers,numstyles,numartno,numbatches,cus6,cus6ps,cus1share
4,A-1,5,1042253.3,117,8908.147863,0.0,Rhino,30698.0,NaN,NaN,...,21.460913,21.460913,21.460913,1.0,10.0,5.0,28.0,NaN,NaN,100.0


## `*ch` (pct change)

In [7]:
wages['pschange']=wages.groupby('conveyor')['allps'].pct_change()*100
wages['stylesch']=wages.groupby('conveyor')['numstyles'].pct_change()*100
wages['artnoch']=wages.groupby('conveyor')['numartno'].pct_change()*100

wages

,conveyor,month,all_salary,workers,avg_pay,attrition,cus1,cus1ps,cus2,cus2ps,...,numbuyers,numstyles,numartno,numbatches,cus6,cus6ps,cus1share,pschange,stylesch,artnoch
0,A-1,1,1119218.3,126,8882.684921,NaN,Curfu,14562.0,Bull,5832.0,...,4.0,13.0,9.0,20.0,NaN,NaN,58.665700,NaN,NaN,NaN
1,A-1,2,1059743.7,114,9295.997368,0.095238,Rhino,15436.0,Curfu,4114.0,...,3.0,16.0,9.0,33.0,NaN,NaN,73.792906,-15.727983,23.076923,0.000000
2,A-1,3,1042766.2,117,8912.531624,0.000000,Curfu,18015.0,Rhino,12421.0,...,3.0,14.0,8.0,29.0,NaN,NaN,57.281399,50.348982,-12.500000,-11.111111
3,A-1,4,1003444.4,116,8650.382759,0.008547,Rhino,29389.0,Curfu,2766.0,...,3.0,17.0,9.0,64.0,NaN,NaN,91.091963,2.585056,21.428571,12.500000
4,A-1,5,1042253.3,117,8908.147863,0.000000,Rhino,30698.0,NaN,NaN,...,1.0,10.0,5.0,28.0,NaN,NaN,100.000000,-4.850758,-41.176471,-44.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,C-6,9,298253.1,38,7848.765789,0.173913,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
126,C-7,6,358132.7,48,7461.097917,NaN,Crown,6681.0,Rhino,144.0,...,2.0,5.0,5.0,5.0,NaN,NaN,97.890110,NaN,NaN,NaN
127,C-7,7,384174.2,49,7840.289796,0.000000,Crown,8390.0,Rhino,906.0,...,3.0,15.0,9.0,19.0,NaN,NaN,90.137516,36.380952,200.000000,80.000000
128,C-7,8,400679.3,52,7705.371154,0.000000,Crown,8760.0,Sparky,1500.0,...,2.0,12.0,6.0,15.0,NaN,NaN,85.380117,10.227761,-20.000000,-33.333333


## export

In [8]:
wages.to_csv('wages_final.csv',index=0)